## Import all the Dependencies

In [ ]:
!pip install tensorflow

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
import os

2024-12-30 21:06:52.534494: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'keras.src'

## Set all the Constants

In [2]:

BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=5

## Import data into tensorflow dataset object

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
minidataset = "/Users/edwinblanco/Desktop/ML_projects/Coffee-Disease-Prediction/Dataset/Mini Dataset "

In [ ]:
minidatasetColab = "/content/drive/MyDrive/Colab Notebooks/Mini Dataset "

In [5]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    minidataset,
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 4579 files belonging to 3 classes.


## CNN Model Architecture

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet

def build_model():
    base_model = MobileNet(weights='imagenet', include_top=False)  # Load MobileNet as base model

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # Add dense layers to make the model learn more complex functions
    x = Dense(1024, activation='relu')(x)  # Dense layer 1
    x = Dense(1024, activation='relu')(x)  # Dense layer 2
    x = Dense(512, activation='relu')(x)   # Dense layer 3
    preds = Dense(3, activation='softmax')(x)  # Final layer with softmax activation for 3 classes

    # Create the model instance tying together the input of the base model and the output of the final layer
    model = Model(inputs=base_model.input, outputs=preds)

    return model


## Train Test Split

In [7]:
datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(
            validation_split=0.2)
datagen_test = tf.keras.preprocessing.image.ImageDataGenerator(
            validation_split=0.2)

In [8]:
os.mkdir("test")
os.mkdir("train")

In [9]:
for i, one_class in enumerate(os.listdir(minidataset)):
    print("Test Class",one_class)
    os.mkdir("./test/"+one_class)
    gen = datagen_test.flow_from_directory(
        minidataset,
        target_size = (224, 224),
        batch_size = 1,
        classes = [one_class],
        save_to_dir = f'./test/{one_class}',#this allows it to go to the path where the images will be saved, they should be created before which is why we have the chdir
        save_prefix = 'new_image',
        save_format = 'jpg',
        seed=1,#seed is to be able to reproduce the split. If we have the same seed, then the data will be split in the same way everytime
        subset="validation",#this is to say that this will be the validation or test part, which will take a 0.2 partition of the data as specified in datagen_test and datagen_train
        shuffle=True#shuffle is true to be able to create different splits
        )
    for j in range(0,len(gen)):
                gen.next()

Test Class .DS_Store
Found 0 images belonging to 1 classes.
Test Class Healthy
Found 308 images belonging to 1 classes.


AttributeError: 'DirectoryIterator' object has no attribute 'next'

In [ ]:
for i, one_class in enumerate(os.listdir(minidataset)):
        os.mkdir("./train/"+one_class)
        print("Train Class",one_class)
        gen = datagen_test.flow_from_directory(
                minidataset,
                target_size = (224, 224),
                batch_size = 1,
                classes = [one_class],
                save_to_dir = f'./train/{one_class}',
                save_prefix = 'new_image',
                save_format = 'jpg',
                seed=1,
                subset="training",#this is to say that this will be the training part, which will take a 0.2 partition of the data as specified in datagen_test and datagen_train
                shuffle=True
              )
        for j in range(0,len(gen)):
                gen.next()

Train Class Healthy
Found 1232 images belonging to 1 classes.
Train Class Rust
Found 1235 images belonging to 1 classes.
Train Class Miner
Found 1201 images belonging to 1 classes.


## Train the model

In [ ]:
datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            validation_split=0.2,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest')

In [ ]:
datagen_test = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255)

In [ ]:
model=build_model()

#compile the model, train it, and then reset it
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

/var/folders/9c/5g9mgmdj15g6g0rh13sjypdh0000gn/T/ipykernel_71728/256648939.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(weights='imagenet', include_top=False)  # Load MobileNet as base model


In [ ]:
train_generator = datagen_train.flow_from_directory(
    "./train",
    seed=1,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset='training')#reading the training data


PermissionError: [Errno 1] Operation not permitted: './train'

In [ ]:

test_generator = datagen_test.flow_from_directory(
    "./test",
    seed=1,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset='validation')

Found 0 images belonging to 3 classes.


In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:

history=model.fit(
    train_generator,
    epochs=3,
    verbose=1,
    steps_per_epoch=train_generator.samples//train_generator.batch_size)#fitting the data


Epoch 1/5
91/91 [==============================] - 791s 9s/step - loss: 0.1642 - accuracy: 0.9521
Epoch 2/5
91/91 [==============================] - 773s 8s/step - loss: 0.0220 - accuracy: 0.9938
Epoch 3/5
91/91 [==============================] - 767s 8s/step - loss: 0.0517 - accuracy: 0.9855
Epoch 4/5
91/91 [==============================] - 776s 9s/step - loss: 0.1021 - accuracy: 0.9773
Epoch 5/5
91/91 [==============================] - 765s 8s/step - loss: 0.0562 - accuracy: 0.9879


In [ ]:
os.mkdir("/content/drive/MyDrive/Colab Notebooks/saved_models")

In [ ]:
import os
model_version=max([int(i) for i in os.listdir("/content/drive/MyDrive/Colab Notebooks/saved_models") + [0]])+1
model.save(f"/content/drive/MyDrive/Colab Notebooks/saved_models/{model_version}")

In [ ]:
os.getcwd()

'/content'